We would needd `smmregrid` for that (https://github.com/jhardenberg/smmregrid/tree/main)

In [3]:
import smmregrid as rg

In [4]:
import xarray as xr
import numpy as np
import matplotlib.pylab as plt
import matplotlib.cm as cm
import glob
import intake
import os

In [5]:
cat = intake.open_catalog("https://nextgems.github.io/catalog/catalog.yaml")

In [6]:
experiment = 'IFS_4.4-FESOM_5-cycle3'

In [7]:
print(list(cat.FESOM[experiment]))

['2D_1h_native', '2D_daily_native', '2D_monthly_native', '3D_3h_native', '3D_daily_native', 'elem_grid', 'node_grid']


In [14]:
what_interpolate = '2D_daily_native'
output_folder = "/work/bm1344/AWI/Cycle3/"
start_year = 2020
end_year = 2023
resolution = '025'

In [15]:
data = cat.FESOM[experiment][what_interpolate].to_dask()

In [16]:
data

<xarray.Dataset>
Dimensions:   (time: 1442, nod2: 7402886)
Coordinates:
  * time      (time) datetime64[ns] 2020-01-20T23:56:00 ... 2023-12-31T23:56:00
Dimensions without coordinates: nod2
Data variables: (12/13)
    MLD1      (time, nod2) float32 dask.array<chunksize=(1, 7402886), meta=np.ndarray>
    MLD2      (time, nod2) float32 dask.array<chunksize=(1, 7402886), meta=np.ndarray>
    atmice_x  (time, nod2) float32 dask.array<chunksize=(1, 7402886), meta=np.ndarray>
    atmice_y  (time, nod2) float32 dask.array<chunksize=(1, 7402886), meta=np.ndarray>
    atmoce_x  (time, nod2) float32 dask.array<chunksize=(1, 7402886), meta=np.ndarray>
    atmoce_y  (time, nod2) float32 dask.array<chunksize=(1, 7402886), meta=np.ndarray>
    ...        ...
    m_ice     (time, nod2) float32 dask.array<chunksize=(1, 7402886), meta=np.ndarray>
    m_snow    (time, nod2) float32 dask.array<chunksize=(1, 7402886), meta=np.ndarray>
    prec      (time, nod2) float32 dask.array<chunksize=(1, 7402886), meta=np.ndarray>
    ssh       (time, nod2) float32 dask.array<chunksize=(1, 7402886), meta=np.ndarray>
    uice      (time, nod2) float32 dask.array<chunksize=(1, 7402886), meta=np.ndarray>
    vice      (time, nod2) float32 dask.array<chunksize=(1, 7402886), meta=np.ndarray>
Attributes: (12/17)
    FESOM_model:                         FESOM2
    FESOM_website:                       fesom.de
    FESOM_git_SHA:                       dfbf523
    FESOM_MeshPath:                      /work/bm1235/ifs-inputs_nxg_c3/fesom...
    FESOM_mesh_representative_checksum:  
    FESOM_ClimateDataPath:               /work/bm1235/ifs-inputs_nxg_c3/fesom...
    ...                                  ...
    FESOM_use_floatice:                  0
    FESOM_whichEVP:                      1
    FESOM_evp_rheol_steps:               120
    FESOM_opt_visc:                      5
    FESOM_use_wsplit:                    -1
    FESOM_autorotate_back_to_geo:        -1

In [8]:
from dask.distributed import Client
import dask
dask.config.config.get('distributed').get('dashboard').update({'link':'{JUPYTERHUB_SERVICE_PREFIX}/proxy/{port}/status'})

client = Client(n_workers=20, threads_per_worker=1, memory_limit='10GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/a270088/levante-spawner-preset//proxy/8787/status,
Dashboard: /user/a270088/levante-spawner-preset//proxy/8787/status,Workers: 20
Total threads: 20,Total memory: 186.26 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42019,Workers: 20
Dashboard: /user/a270088/levante-spawner-preset//proxy/8787/status,Total threads: 20
Started: Just now,Total memory: 186.26 GiB
Comm: tcp://127.0.0.1:42399,Total threads: 1
Dashboard: /user/a270088/levante-spawner-preset//proxy/34113/status,Memory: 9.31 GiB
Nanny: tcp://127.0.0.1:38419,


In [9]:
regridder = rg.Regridder(weights=f"/work/ab0995/a270088/NextGems/Cycle3/weights/weights_FESOM_tco2559-ng5_original_2d_ycon_ecmwf_{resolution}_l2d.nc")

In [10]:
def ensure_directory_exists(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        print(f"Directory {dir_path} created ")
    else:    
        print(f"Directory {dir_path} already exists")
def check_file_exists(filepath):
    return os.path.isfile(filepath)

In [ ]:
for var_name, da in data.data_vars.items():
    opath = os.path.join(output_folder, "FESOM", experiment, resolution, what_interpolate, var_name)
    ensure_directory_exists(opath)
    for year in range(start_year, end_year+1):
        for mon in range(1, 13):
            print(f"{var_name}, {year}, {mon}")
            ofile = os.path.join(opath, f"{var_name}_{str(year)}-{str(mon).zfill(2)}.nc")
            if not check_file_exists(ofile):
                monthly_interp = regridder.regrid(da.sel(time=f'{str(year)}-{str(mon).zfill(2)}'))
                monthly_interp = monthly_interp.to_dataset()
                monthly_interp.attrs = {'model': "FESOM2",
                                        'experiment':experiment,
                                        'var_type': what_interpolate, 
                                        "target_resolution": resolution}
                monthly_interp.to_netcdf(ofile, encoding={var_name: {"zlib": True, "complevel": 1, "dtype": np.dtype("single")}},)
            else:
                print(f"File {ofile} exist, skipping")
                continue
            
            

In [ ]:
# cat = intake.open_catalog("https://nextgems.github.io/catalog/catalog.yaml")